In [1]:
state_to_id = {
    'Alabama': 1,
    "Alaska":2,
    "Arizona":4,
    "Arkansas":5,
    "California":6,
    "Colorado":8,
    "Connecticut":9,
    "Delaware":10,
    "District of Columbia": 11,
    "Florida":12,
    "Georgia":13,
    "Hawaii":15,
    "Idaho":16,
    "Illinois":17,
    "Indiana":18,
    "Iowa":19,
    "Kansas":20,
    "Kentucky":21,
    "Louisiana":22,
    "Maine":23,
    "Maryland":24,
    "Massachusetts":25,
    "Michigan":26,
    "Minnesota":27,
    "Mississippi":28,
    "Missouri":29,
    "Montana":30,
    "Nebraska":31,
    "Nevada":32,
    "New Hampshire":33,
    "New Jersey":34,
    "New Mexico":35,
    "New York":36,
    "North Carolina":37,
    "North Dakota":38,
    "Ohio":39,
    "Oklahoma":40,
    "Oregon":41,
    "Pennsylvania":42,
    "Rhode Island":44,
    "South Carolina":45,
    "South Dakota":46,
    "Tennessee":47,
    "Texas":48,
    "Utah":49,
    "Vermont":50,
    "Virginia":51,
    "Washington":53,
    "West Virginia":54,
    "Wisconsin":55,
    "Wyoming":56,
    "Puerto Rico":72,
}

In [2]:
import pandas as pd
import altair as alt
from vega_datasets import data
alt.renderers.set_embed_options(theme='light')

df = pd.read_csv('interactive.csv')
for i, row in df.iterrows():
    df.at[i, 'state'] = ' '.join([word.capitalize() for word in row['state'].split(' ')])
    if df.at[i, 'state'] == 'District Of Columbia':
        df.at[i, 'state'] = 'District of Columbia'
for i, row in df.iterrows():
    df.at[i, 'id'] = int(state_to_id[row['state']])
df.head()

,state,income,year,id
0,Alabama,39400,2019,1.0
1,Alaska,62600,2019,2.0
2,Arizona,49400,2019,4.0
3,Arkansas,35700,2019,5.0
4,California,63300,2019,6.0


In [3]:
state_map_url = data.us_10m.url
state_map = alt.topo_feature(state_map_url, 'states')

state_clicker = alt.selection_multi(fields=['state'])
slider = alt.binding_range(min=2008, max=2019, step=1, name='Slide to choose the year: ')
year_selector = alt.selection_single(
    fields=['year'],
    bind=slider,
    init={'year': 2019}
)


color_scale=alt.Scale(
    domain=[25000, 35000, 45000, 55000, 65000],
    range=['red','orange','white','lightblue','darkblue'],
    interpolate='hcl'
)


choropleth = alt.Chart(df).mark_geoshape(
    stroke='black',
    strokeWidth=0.5,
).encode(
    tooltip=['state:N', 'income:Q'],
    color=alt.Color(
        'income:Q',
        scale=color_scale,
        legend=alt.Legend(
            orient='none',
            legendX=425, legendY=200,
            titleAnchor='start',
        )
    ),
    opacity=alt.condition(state_clicker, alt.value(1), alt.value(0.2))
).add_selection(
    state_clicker,
    year_selector
).transform_filter(
    year_selector
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(state_map, 'id', fields=["type", "properties", "geometry"])
).project(
    type='albersUsa'
).properties(
    width=600,
    height=600
)

bars = alt.Chart(df).mark_bar(
    size=8
).encode(
    x=alt.X('income:Q', axis=alt.Axis(title='Year', labelAngle=-45), scale=alt.Scale(domain=[0, df.income.max()])),
    y=alt.Y('state:N', sort='x'),
    opacity=alt.condition(state_clicker, alt.value(1), alt.value(0.2)),
    color=alt.Color(
        'income:Q',
        scale=color_scale,
        legend=None
    ),
    tooltip=['state:N', 'income:Q'],
).add_selection(
    state_clicker
).transform_filter(
    year_selector
).transform_window(
    rank='rank(income)',
    sort=[alt.SortField('income', order='ascending')]
).transform_filter(
    alt.datum.rank <= 100 # for top k elements if want
).properties(
    width=400,
    height=600
)

lines = alt.Chart(df).transform_filter(
    year_selector
).transform_aggregate(
    Q1='q1(income):Q',
    median='median(income):Q',
    Q3='q3(income):Q',
).transform_fold(
    ['Q1', 'median', 'Q3']
).mark_rule(
    strokeWidth=3
).encode(
    x='value:Q',
    color=alt.Color(
        'key:N',
        legend=alt.Legend(
            title='Statistics'
        ),
        scale=alt.Scale(range=['#ffbb9e','#57A44C','#9ecdff'])
    ),
    tooltip=[alt.Tooltip('key:N', title="attribute"), alt.Tooltip('value:O', title="income")]
)

chart = ((bars + lines) | choropleth).properties(
    title={
        'text': 'Median Annual Household Income of Working Age Disabled People with a Disability',
        'fontSize': 24
    }
).configure_title(
    anchor='middle'
)

from IPython.display import HTML
display(
    HTML(
        """
            <style>
                .vega-bind {
                  text-align:center;
                }
            </style>
        """
    )
)
display(chart)


alt.HConcatChart(...)